<a href="https://colab.research.google.com/github/waltz2u/HuskyCoin/blob/master/Lense_Classification_using_Vanilla_Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
##################
# Decision Trees #
##################
import numpy as np

class decision_tree:
    def __init__(self):
        self.split_value = 0
        self.split_attr = -1
        self.max_gain = 0.1
        self.left = None
        self.right = None
        self.depth = 0
        self.distribution = []

def entropy(data, target_attr):
    # Calculates the entropy of the given data set for the target attribute.
    val_freq = {}
    data_entropy = 0.0
 
    # Calculate the frequency of each of the values in the target attr
    for record in data:
        if (record[target_attr]) in val_freq.keys():
            val_freq[record[target_attr]] += 1.0
        else:
            val_freq[record[target_attr]]  = 1.0
 
    # Calculate the entropy of the data for the target attribute
    for freq in val_freq.values():
        data_entropy += (-freq/len(data)) * math.log(freq/len(data), 2) 
        
    return data_entropy

def gain(data, attr, target_attr):
    # Calculates the information gain (reduction in entropy) that would result by splitting the data on the chosen attribute (attr).
    val_freq = {}
    subset_entropy = 0.0
 
    # Calculate the frequency of each of the values in the target attribute
    for record in data:
        if (record[attr] in val_freq):
            val_freq[record[attr]] += 1.0
        else:
            val_freq[record[attr]]  = 1.0
 
    # Calculate the sum of the entropy for each subset of records weighted by their probability of occuring in the training set.
    for val in val_freq.keys():
        val_prob = val_freq[val] / sum(val_freq.values())
        data_subset = [record for record in data if record[attr] == val]
        subset_entropy += val_prob * entropy(data_subset, target_attr)
 
    # Subtract the entropy of the chosen attribute from the entropy of the whole data set with respect to the target attribute (and return it)
    return (entropy(data, target_attr) - subset_entropy)

def growTree(node, data,target_attr):
    # which attribute and value is best to split
    for i in range(len(data)): # go over each instance to split to left/right
        for j in range(len(data[0])): # in each instance, go over each attribute
            if gain(data,j,target_attr) > node.max_gain and j != target_attr:
                node.split_value = data[i][j]
                node.split_attr = j
                node.max_gain = gain(data,j,target_attr)

    if node.split_attr != -1:           
        
        print("=" * node.depth + "v" + str(node.split_attr) + ">" + str(node.split_value))
                
        # now split data
        left_data, right_data = [], []
        for i in range(len(data)):
            if data[i][node.split_attr] > node.split_value:
                right_data.append(data[i])
            else:
                left_data.append(data[i])
        node.distribution = {i:[row[-1] for row in data].count(i) for i in set([row[-1] for row in data])}
    if node.split_attr == -1 or len(data) <2 or len(right_data) <1 or len(left_data) <1:
        # no more splitting
        node.leaf = True
        node.distribution = {i:[row[-1] for row in data].count(i) for i in set([row[-1] for row in data])}
        # max(dic, key=dic.get) returns the key that has max value in a dict namely dic
        print("=" * node.depth + "leaf: classify to class=" + str(max(node.distribution, key=node.distribution.get)) + " with class distribution " + "[" + str(node.distribution) + "]")
        return node

    node.left = decision_tree()
    node.left.depth = node.depth + 1
    node.right = decision_tree()
    node.right.depth = node.depth + 1
    growTree(node.left, left_data, target_attr)
    growTree(node.right, right_data, target_attr)

def classify(node, X):
    # classify test instances in X given a tree
    n = np.size(X, 0)
    pred = np.zeros(n)
    for i in range(n):
        while True:
            if not(node.left == None and node.right == None):
                if X[i][node.split_attr] >= node.split_value: # if the value of the ith instance is >= split value, 
                    node = node.right
                else:
                    node = node.left
            else:
                #if is_int(max(node.distribution, key=node.distribution.get)):
                #    pred[i] = random.choice(set(node.distribution)) #random choice if distribution is equal
                #else:
                pred[i] = max(node.distribution, key=node.distribution.get)
                break
    return(pred)
    
import math
# Lenses data https://archive.ics.uci.edu/ml/machine-learning-databases/lenses/lenses.data
data = [[1, 1, 1, 1, 1, 3],
[2, 1, 1, 1, 2, 2],
[3, 1, 1, 2, 1, 3],
[4, 1, 1, 2, 2, 1],
[5, 1, 2, 1, 1, 3],
[6, 1, 2, 1, 2, 2],
[7, 1, 2, 2, 1, 3],
[8, 1, 2, 2, 2, 1],
[9, 2, 1, 1, 1, 3],
[10, 2, 1, 1, 2, 2],
[11, 2, 1, 2, 1, 3],
[12, 2, 1, 2, 2, 1],
[13, 2, 2, 1, 1, 3],
[14, 2, 2, 1, 2, 2],
[15, 2, 2, 2, 1, 3],
[16, 2, 2, 2, 2, 3],
[17, 3, 1, 1, 1, 3],
[18, 3, 1, 1, 2, 3],
[19, 3, 1, 2, 1, 3],
[20, 3, 1, 2, 2, 1],
[21, 3, 2, 1, 1, 3],
[22, 3, 2, 1, 2, 2],
[23, 3, 2, 2, 1, 3],
[24, 3, 2, 2, 2, 3]]

tree = decision_tree()
growTree(tree,[row[1:] for row in data],4)
tree

trainpred = classify(tree, data)
yTrain = [row[-1] for row in data]
print("Train Accuracy: %f\n" % (np.mean(yTrain == trainpred) * 100))

v3>1
=leaf: classify to class=3 with class distribution [{3: 12}]
=v2>1
==v0>1
===leaf: classify to class=2 with class distribution [{2: 2}]
===v0>2
====leaf: classify to class=2 with class distribution [{2: 2}]
====v1>1
=====leaf: classify to class=3 with class distribution [{3: 1}]
=====leaf: classify to class=2 with class distribution [{2: 1}]
==v1>1
===leaf: classify to class=1 with class distribution [{1: 3}]
===v0>1
====leaf: classify to class=1 with class distribution [{1: 1}]
====leaf: classify to class=3 with class distribution [{3: 2}]
Train Accuracy: 62.500000

